In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

In [10]:
df = pd.read_csv(r"C:\Mine\FreeCodeCamp Web Scrapping\wuzzuf_data_analyst.csv")
df.head()

,job_title,company_name,location,duration,job_type,job_status,level,years_of_experience,skills_required
0,Financial Data Analyst,ECC Group,"Nasr City, Cairo, Egypt",4 days ago,Full Time,On-site,Experienced,3 - 5,Experienced · 3 - 5 Yrs of Exp · Education/Tea...
1,Manager - Data Intelligence Analyst,Almosafer,"Mohandessin, Giza, Egypt",6 days ago,Full Time,On-site,Manager,5 - 8,Manager · 5 - 8 Yrs of Exp · Analyst/Research ...
2,Supply Chain Data Analyst,ZeroCarbon,"Cairo, Egypt",7 days ago,Full Time,On-site,Experienced,2 - 3,Experienced · 2 - 3 Yrs of Exp · Logistics/Sup...
3,Data Analyst,Specialized Seamless Services,"Heliopolis, Cairo, Egypt",3 days ago,Full Time,On-site,Entry Level,2 - 4,Entry Level · 2 - 4 Yrs of Exp · Administratio...
4,Data Analyst,Venus Hiring,"Toronto, Canada",13 days ago,Full Time,On-site,Entry Level,1 - 5,Entry Level · 1 - 5 Yrs of Exp · Analyst/Resea...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_title            219 non-null    object
 1   company_name         219 non-null    object
 2   location             219 non-null    object
 3   duration             219 non-null    object
 4   job_type             219 non-null    object
 5   job_status           219 non-null    object
 6   level                219 non-null    object
 7   years_of_experience  219 non-null    object
 8   skills_required      219 non-null    object
dtypes: object(9)
memory usage: 15.5+ KB


## Major modifications
#### 1. `Duration` column:
- Create new column called date.
- Calculate the date when the job was posted.
- Remove any unvalid data from the column.
- Delete `duration` column.

#### 2. `Years of Experience` column
- Replace all the invalid values with NaN.

#### 3. `Skills Required` column:
- Remove extra text from the cells.
- Put the skills into lists.


In [12]:
df[["num_days", "day_month", "ago"]]=df.duration.str.split(expand=True)
df[["num_days", "day_month", "ago"]]

,num_days,day_month,ago
0,4,days,ago
1,6,days,ago
2,7,days,ago
3,3,days,ago
4,13,days,ago
...,...,...,...
214,2,months,ago
215,2,months,ago
216,7,days,ago
217,7,days,ago


In [13]:
df.num_days = df.num_days.astype(int)

In [14]:
df["days"] = df.apply(lambda x: x["num_days"]*30 if x["day_month"] == "months" else x["num_days"], axis=1)

In [15]:
df["month_posted"] = df.apply(lambda x: (datetime.today() - timedelta(days=x["days"])).strftime("%B"), axis=1)

In [16]:
df.drop(columns=["num_days", "day_month", "ago", "days", "duration"], inplace=True)

In [17]:
df[["exp_from", "exp_to"]]=df["years_of_experience"].str.split(" - ", expand=True)
df[["exp_from", "exp_to"]]

,exp_from,exp_to
0,3,5
1,5,8
2,2,3
3,2,4
4,1,5
...,...,...
214,3,5
215,4,6
216,"Riyadh, Saudi Arabia",None
217,"Doha, Qatar",None


In [18]:
df.exp_to.unique()

array(['5 ', '8 ', '3 ', '4 ', None, '15 ', '7 ', '9 ', '6 ', '1 ', '2 ',
       '10 ', '20 ', '13 ', '12 ', '14 '], dtype=object)

In [19]:
df["experience"] = df.apply(lambda x: x.exp_from + "-" + x.exp_to if len(x.exp_from) <= 2 else "Not specified", axis=1)
df["experience"]

0               3-5 
1               5-8 
2               2-3 
3               2-4 
4               1-5 
           ...      
214             3-5 
215             4-6 
216    Not specified
217    Not specified
218    Not specified
Name: experience, Length: 219, dtype: object

In [20]:
df.drop(columns=["years_of_experience", "exp_from", "exp_to"], inplace=True)

In [21]:
df["skills_required"] = df["skills_required"].map(lambda x: re.sub(r'^.*Yrs of Exp\s*·\s*', '', x))

In [22]:
df["skills_required"] = df["skills_required"].str.split(" · ")

In [23]:
df

,job_title,company_name,location,job_type,job_status,level,skills_required,month_posted,experience
0,Financial Data Analyst,ECC Group,"Nasr City, Cairo, Egypt",Full Time,On-site,Experienced,"[Education/Teaching, Data, CFA, Analyst, Finance]",March,3-5
1,Manager - Data Intelligence Analyst,Almosafer,"Mohandessin, Giza, Egypt",Full Time,On-site,Manager,"[Analyst/Research, Tourism/Travel, Power BI, Q...",March,5-8
2,Supply Chain Data Analyst,ZeroCarbon,"Cairo, Egypt",Full Time,On-site,Experienced,"[Logistics/Supply Chain, Analyst/Research, bus...",March,2-3
3,Data Analyst,Specialized Seamless Services,"Heliopolis, Cairo, Egypt",Full Time,On-site,Entry Level,"[Administration, Analyst/Research, Microsoft O...",March,2-4
4,Data Analyst,Venus Hiring,"Toronto, Canada",Full Time,On-site,Entry Level,"[Analyst/Research, Engineering - Telecom/Techn...",March,1-5
...,...,...,...,...,...,...,...,...,...
214,Tax Accountant,DC Auto,"Maadi, Cairo, Egypt",Full Time,On-site,Experienced,"[Accounting/Finance, Accounting, accountant, T...",January,3-5
215,Financial Planning And Analysis Head,Easy Group For Health and Beauty Care,"Alexandria, Egypt",Full Time,On-site,Experienced,"[Accounting/Finance, Business Development, Ana...",January,4-6
216,Senior ETL Developer,Master-Works,"Riyadh, Saudi Arabia",Full Time,Not specified,Not specified,"[Not specified, Business Development, IT/Softw...",March,Not specified
217,Senior Report Developer,El Seif Engineering Contracting Company,"Riyadh, Saudi Arabia",Full Time,Not specified,Not specified,"[Not specified, Business Development, IT/Softw...",March,Not specified


In [24]:
df = df[["job_title", "job_type", "company_name", "location", "job_status","level", "experience", "month_posted", "skills_required"]]
df.head()

,job_title,job_type,company_name,location,job_status,level,experience,month_posted,skills_required
0,Financial Data Analyst,Full Time,ECC Group,"Nasr City, Cairo, Egypt",On-site,Experienced,3-5,March,"[Education/Teaching, Data, CFA, Analyst, Finance]"
1,Manager - Data Intelligence Analyst,Full Time,Almosafer,"Mohandessin, Giza, Egypt",On-site,Manager,5-8,March,"[Analyst/Research, Tourism/Travel, Power BI, Q..."
2,Supply Chain Data Analyst,Full Time,ZeroCarbon,"Cairo, Egypt",On-site,Experienced,2-3,March,"[Logistics/Supply Chain, Analyst/Research, bus..."
3,Data Analyst,Full Time,Specialized Seamless Services,"Heliopolis, Cairo, Egypt",On-site,Entry Level,2-4,March,"[Administration, Analyst/Research, Microsoft O..."
4,Data Analyst,Full Time,Venus Hiring,"Toronto, Canada",On-site,Entry Level,1-5,March,"[Analyst/Research, Engineering - Telecom/Techn..."


In [25]:
df.month_posted.unique()

array(['March', 'February', 'January'], dtype=object)

In [ ]:
df.to_excel("C:\Mine\FreeCodeCamp Web Scrapping\wuzzuf_data_analyst_Q1_2025.xlsx", index=False, header=True)

In [31]:
with open(r"C:\Mine\FreeCodeCamp Web Scrapping\Confirmation.txt", "w") as file:
    file.write(f"Data size: {len(df)}" + "/n")
    file.write(f"Months jobs published: {df["month_posted"].unique()}")